In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
plt.ioff()
import numpy as np
    
from smfret_analysis import Analyzer, DensityPlots, print_info

In [2]:
print_info()

smFRET analysis software version 2.1
(git revision 2.1-87-ga66b93a)
Output version 13
Using sdt-python version 15.2


In [3]:
# Create an Analyzer instance. This will load the tracking data.
ana = Analyzer("tracking-split")

In [4]:
# Show apparent E vs. S plots after filtering steps. Here we use the JupyterLab
# Sidecar class (https://github.com/jupyter-widgets/jupyterlab-sidecar) for 
# display, but it would also possible to output to a normal notebook cell.

# Which datasets to plot.
plot_keys = []  # Use empty list to disable plotting.
# plot_keys = [k for k in ana.analyzers if k.endswith(" cells")]  # Plot datasets with cells

if plot_keys:
    from sidecar import Sidecar
    sc = Sidecar(title="Filtered plots")
    
    dp = DensityPlots(ana, plot_keys)
    with sc:
        display(dp)

In [5]:
# Plot initial data

# Due to typically large amounts of data, this may take a few tens of seconds
if plot_keys:
    ana.calc_fret_values(a_mass_interp="next")
    dp.dscatter("Initial")  # Scatter plot
    # dp.contourf("Initial")  # Contour plot
    # dp.colormesh("Initial")  # Continous color

In [6]:
# Display laser beam profile and find threshold of brightest xx %
# to ensure good SNR
ana.find_beam_shape_thresh()

In [7]:
# Copy values from above to remove poorly illuminated datapoints
# Manually copy the parameters from above here.
ana.filter_beam_shape_region("donor", 50)

if plot_keys:
    dp.dscatter("Beam shape")

In [8]:
# Allow only tracks with with a high number (here, >75%) of datapoints where
# PSFs don't overlap
ana.query_particles("fret_has_neighbor == 0", min_rel=0.75)
# For the remaining tracks, allow only those datapoints with non-overlapping
# PSFs
ana.query("fret_has_neighbor == 0")
# Accept only tracks that are already visible in the beginning to record 
# full bleaching profile
ana.present_at_start()

if plot_keys:
    dp.dscatter("At start")

In [9]:
# Apply flatfield correction in both channels
ana.flatfield_correction()

In [10]:
# Calculate apparent FRET values, e.g., efficiency and stoichiometry
ana.calc_fret_values(a_mass_interp="next")

In [11]:
# Find thresholds for changepoint detection in intensity upon donor
# excitation (d_mass) as well as upon acceptor excitation (a_mass)
# Typically, one would find first the correct order of magnitude, then
# fine-tune.
ana.find_segment_options()

In [12]:
# Segment tracks using changepoint detection in the intensity time trace
# upon donor excitation and upon acceptor excitation.
# Manually copy the parameters from above here.
ana.segment_mass("donor", penalty=150000000)
ana.segment_mass("acceptor", penalty=100000000)

In [13]:
# Remove trajectories where acceptor does not bleach in a single step and
# donor shows partial bleaching.
# Parameters describe maximum intensity which is considered bleached for
# donor and acceptor.
ana.filter_bleach_step(1000, 500)

if plot_keys:
    dp.dscatter("Bleaching steps")

In [14]:
# The excitation efficiency factor is computer from data with known 1:1
# stoichiomtry. The call below allows for selecting a dataset and for fitting
# Gaussian mixture models to select the right component in an E–S plot.
# Parameters should be passed to calc_excitation_eff() in the cell below.
ana.find_excitation_eff_component()

In [15]:
# Correction coefficients

# If determined in a separate experiment, set leakage and direct exc
# like below. Detection and excitation efficiency can be set similarly.
ana.set_leakage(0.07920904893381935)
ana.set_direct_excitation(0.04464781911255539)

# If donor-only and acceptor-only data is included in the datasets
# and have the correct category set using Tracker.add_dataset,
# the following can be used to calculate leakage and direct excitation
# coefficients
# ana.calc_leakage()
# ana.calc_direct_excitation()

# If you don't have donor-only and acceptor-only samples, leakage and direct
# excitation can can be calculated from those parts of traces that have one
# fluorophore bleached. This may not work so well for the direct excitation
# factor if the donor fluorophore is much more bleach-resistant than the 
# acceptor. Setting print_summary=True will print the result as well as the
# number of datapoints used.
# ana.calc_leakage_from_bleached(print_summary=True)
# ana.calc_direct_excitation_from_bleached(print_summary=True)

# Calculate detection efficiency (gamma) factor from donor and acceptor
# intensity differences upon acceptor bleaching using the specified dataset.
ana.calc_detection_eff(min_part_len=5, how=np.nanmedian, dataset="DPPC control no-cells")

# Calculate excitation efficiency factor using the specified dataset and
# component from a Gaussian mixture model fit (see cell above for an
# interactive interface to find correct settings).
ana.calc_excitation_eff("DPPC control no-cells", n_components=1, component=0)

In [16]:
# Apply correction factors to calculate real FRET efficiencies and 
# stoichiometries
ana.fret_correction()

In [17]:
# Select only data before bleaching of a fluorophore, i.e., where the segment
# number of both donor excitation and acceptor excitation intensity time traces
# is 0. Select only data recorded during donor ("d") excitation.
ana.query("fret_d_seg == 0 and fret_a_seg == 0 and fret_exc_type == 'd'")
# Select only traces where > 75% of datapoints are within reasonable
# stoichiometry limits
ana.query_particles("0.35 <= fret_stoi <= 0.6", min_rel=0.75)
# Remove data points where donor is gone (should not be necessary due to the
# above)
# ana.query("fret_d_mass > fret_d_mass.median() * 0.2")

if plot_keys:
    dp.dscatter("Before bleaching, correct stoi")

In [18]:
# Display an UI to find thresholding parameters for cell images
ana.find_cell_mask_params()

Thresholder(children=(ImageSelector(children=(Dropdown(description='image', options=(('DPPC_ctrl-J4/cells-01_0…

In [20]:
# Threshold cell images and select only data within cell-occupied areas
ana.apply_cell_masks([k for k in ana.analyzers if not k.endswith("no-cells")],
                     "adaptive", block_size=65, c=-2, smooth=3, method="mean")

if plot_keys:
    dp.dscatter("Underneath cells")

In [21]:
# Save results to disk (typically "filtered-v???.h5")
ana.save()

/home/lukas/Software/fret-analysis/smfret_analysis/analyzer.py:860: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis1_level0] [items->None]

  {("fret", "exc_type"): str}))
